# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/djdinnebeil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/djdinnebeil/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefully be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/home/djdinnebeil/aie7_journey/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/home/djdinnebeil/aie7_journey/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/home/djdinnebeil/aie7_journey/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [8]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [9]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/18 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/27 [00:00<?, ?it/s]

Property 'summary' already exists in node '7b2ebf'. Skipping!
Property 'summary' already exists in node 'c94fdf'. Skipping!
Property 'summary' already exists in node '39e8da'. Skipping!
Property 'summary' already exists in node 'ae703a'. Skipping!
Property 'summary' already exists in node '4898b4'. Skipping!
Property 'summary' already exists in node '875fd7'. Skipping!
Property 'summary' already exists in node '16fb58'. Skipping!
Property 'summary' already exists in node '37f730'. Skipping!
Property 'summary' already exists in node '06f493'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/18 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/57 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node 'c94fdf'. Skipping!
Property 'summary_embedding' already exists in node '39e8da'. Skipping!
Property 'summary_embedding' already exists in node '7b2ebf'. Skipping!
Property 'summary_embedding' already exists in node '06f493'. Skipping!
Property 'summary_embedding' already exists in node '875fd7'. Skipping!
Property 'summary_embedding' already exists in node '37f730'. Skipping!
Property 'summary_embedding' already exists in node 'ae703a'. Skipping!
Property 'summary_embedding' already exists in node '16fb58'. Skipping!
Property 'summary_embedding' already exists in node '4898b4'. Skipping!


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 44, relationships: 390)

We can save and load our knowledge graphs as follows.

In [10]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 44, relationships: 390)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [11]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [12]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

##### ✅ Answer:

### Query Synthesizers in RAGAS

#### 1. `SingleHopSpecificQuerySynthesizer`
- **Description**: Generates direct, fact-based questions that rely on a single piece of information.
- **Example**: *"What is the capital of France?"*
- **Use Case**: Useful for testing basic fact retrieval and precision.

#### 2. `MultiHopAbstractQuerySynthesizer`
- **Description**: Creates complex questions that require reasoning over multiple facts, using abstract or generalized language.
- **Example**: *"Which European city known for its fashion and art is home to the Eiffel Tower?"*
- **Use Case**: Tests a model’s ability to reason and infer from context without relying on exact matches.

#### 3. `MultiHopSpecificQuerySynthesizer`
- **Description**: Produces multi-step questions that involve reasoning across multiple facts but refer to concrete entities.
- **Example**: *"Which city is the capital of the country where Emmanuel Macron is president?"*
- **Use Case**: Evaluates multi-hop reasoning while maintaining specificity, testing both memory and linkage.



Finally, we can use our `TestSetGenerator` to generate our testset!

In [14]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What does the term FSA refer to in the context...,[Application and Verification Guide Introducti...,FSA refers to the U.S. Department of Education...,single_hop_specifc_query_synthesizer
1,What is FAFSA and why was the section for retu...,[Chapter 1: The Application Process We removed...,FAFSA is the application process for financial...,single_hop_specifc_query_synthesizer
2,What is Federal Tax Information and how is it ...,[electronically due to limitations on access t...,The context mentions Federal Tax Information (...,single_hop_specifc_query_synthesizer
3,What should students be aware of regarding the...,[Note: There will be two active portals that c...,Note: There will be two active portals that co...,single_hop_specifc_query_synthesizer
4,How do I access the FAFSA Partner Portal to re...,[Output Documents After processing is complete...,If your school is not listed on the student's ...,single_hop_specifc_query_synthesizer
5,Considering the availability of active portals...,[<1-hop>\n\nelectronically due to limitations ...,The paper FAFSA forms for incarcerated applica...,multi_hop_abstract_query_synthesizer
6,How does the verification of legal emancipatio...,[<1-hop>\n\nSubmission of a court order or off...,Verification of legal emancipation or guardian...,multi_hop_abstract_query_synthesizer
7,Hwo do application and verification proccesses...,[<1-hop>\n\nApplication and Verification Guide...,The application and verification process for f...,multi_hop_abstract_query_synthesizer
8,"Based on the information provided, how does a ...","[<1-hop>\n\nveteran, or will be one by June 30...","A student's veteran status, confirmed through ...",multi_hop_specific_query_synthesizer
9,How does the information on StudentAid.gov hel...,[<1-hop>\n\nBecause the Department matches the...,The context explains that students must ensure...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [15]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/18 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/28 [00:00<?, ?it/s]

Property 'summary' already exists in node '97f483'. Skipping!
Property 'summary' already exists in node '08e374'. Skipping!
Property 'summary' already exists in node '46d1ef'. Skipping!
Property 'summary' already exists in node 'a3bc53'. Skipping!
Property 'summary' already exists in node '162972'. Skipping!
Property 'summary' already exists in node '871d88'. Skipping!
Property 'summary' already exists in node '5852ba'. Skipping!
Property 'summary' already exists in node '82aa59'. Skipping!
Property 'summary' already exists in node '6072cb'. Skipping!
Property 'summary' already exists in node 'fc8e35'. Skipping!


...

Property 'summary_embedding' already exists in node 'fc8e35'. Skipping!
Property 'summary_embedding' already exists in node '6072cb'. Skipping!
Property 'summary_embedding' already exists in node '97f483'. Skipping!
Property 'summary_embedding' already exists in node '08e374'. Skipping!
Property 'summary_embedding' already exists in node 'a3bc53'. Skipping!
Property 'summary_embedding' already exists in node '46d1ef'. Skipping!
Property 'summary_embedding' already exists in node '5852ba'. Skipping!
Property 'summary_embedding' already exists in node '871d88'. Skipping!
Property 'summary_embedding' already exists in node '82aa59'. Skipping!
Property 'summary_embedding' already exists in node '162972'. Skipping!


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [16]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,"What is the significance of Dec. 27, 2020, in ...",[Application and Verification Guide Introducti...,"Dec. 27, 2020, is the date when the FAFSA Simp...",single_hop_specifc_query_synthesizer
1,What is the role of the FPS in the FAFSA appli...,[The FPS also checks the application for possi...,The FPS checks the application for possible in...,single_hop_specifc_query_synthesizer
2,How do I correctly interpret the ISIR in the F...,[Output Documents After processing is complete...,The ISIR is an output document produced after ...,single_hop_specifc_query_synthesizer
3,How do I correctly fill out the FASFA form and...,[2. The disclosure of their FTI by the IRS to ...,The context provided does not mention the FAFS...,single_hop_specifc_query_synthesizer
4,"How does the FAFSA Simplification Act, particu...",[<1-hop>\n\nApplication and Verification Guide...,"The FAFSA Simplification Act, through the impl...",multi_hop_abstract_query_synthesizer
5,How does the disclosure of Federal Tax Informa...,[<1-hop>\n\n2. The disclosure of their FTI by ...,The disclosure of Federal Tax Information (FTI...,multi_hop_abstract_query_synthesizer
6,How does the application processing of the FAF...,[<1-hop>\n\nThe FPS also checks the applicatio...,The application processing of the FAFSA involv...,multi_hop_abstract_query_synthesizer
7,How do signatures and approval requirements fo...,[<1-hop>\n\n2. The disclosure of their FTI by ...,The signatures and approval requirements for F...,multi_hop_abstract_query_synthesizer
8,How does the Federal Student Aid Information C...,[<1-hop>\n\nBecause the Department matches the...,The Federal Student Aid Information Center (FS...,multi_hop_specific_query_synthesizer
9,How does the ISIR relate to the FTI disclosure...,[<1-hop>\n\n2. The disclosure of their FTI by ...,The ISIR (Institutional Student Information Re...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [17]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [18]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [19]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [21]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [22]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [23]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [24]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [25]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [26]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [27]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available are:\n\n- Direct Subsidized Loans  \n- Direct Unsubsidized Loans  \n- Direct PLUS Loans (including student Federal PLUS Loans and parent PLUS Loans)  \n- Subsidized and Unsubsidized Federal Stafford Loans (made under the FFEL Program before July 1, 2010)  \n- Federal SLS Loans (made under the FFEL Program before July 1, 2010)  \n- Federal PLUS Loans (made under the FFEL Program before July 1, 2010)'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [28]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [29]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:

##### ✅ Answer:

### 🧪 LangSmith Evaluators Overview

#### `qa_evaluator`
- **Purpose**: Checks the factual correctness of the model’s response.
- **Question**: *Is the predicted answer accurate when compared to the reference answer?*
- **Use Case**: Evaluating performance in standard QA tasks.

#### `labeled_helpfulness_evaluator`
- **Purpose**: Assesses how helpful the model’s answer is, based on the correct reference answer.
- **Question**: *Does the response provide useful and relevant information for the user?*
- **Use Case**: Testing how well the model supports the user in getting the right information.

#### `empathy_evaluator`
- **Purpose**: Measures the empathy conveyed in the model’s response.
- **Question**: *Does the response make the user feel understood and emotionally supported?*
- **Use Case**: Evaluating tone, emotional intelligence, and user alignment in sensitive or human-centric contexts.


## LangSmith Evaluation

In [30]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'dear-town-68' at:
https://smith.langchain.com/o/8a526e26-7800-4aae-98b7-c8c0ecc66e3c/datasets/5c5d3f15-0802-4af3-81c0-92b099c02df1/compare?selectedSessions=9d04f0ca-94b0-48d3-ad55-1e80422cbaf2




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,Wht is the Title IV?,I don't know.,None,The context explains that Title IV involves pr...,0,0,0,0.720823,4d38a461-c9c6-493b-ae66-878cef366f7b,41f5ab5c-fbae-44e6-9ce2-7f1381744bb9
1,"Student live with disabled sibling and nephew,...",Based on the context provided:\n\n- For an ind...,None,"Based on the context, the student and their di...",1,1,0,5.773971,d4a1474b-9775-4f77-a4ce-a726935b8c29,13aa7eb4-2520-4c9b-b505-753f5f01871e
2,How does the ISIR relate to the FTI disclosure...,"Based on the provided context, the ISIR (Insti...",None,The ISIR (Institutional Student Information Re...,1,1,0,5.307649,1a24a6c5-daac-4315-97bb-aa801aead613,9a7c1249-a720-47dd-95bc-97cedcbbc6e4
3,How does the Federal Student Aid Information C...,The Federal Student Aid Information Center (FS...,None,The Federal Student Aid Information Center (FS...,1,1,0,2.205721,130a05f8-0a6c-48a3-84d5-a6bf297774ad,27f35041-25d0-4f13-93eb-13bddbaada06
4,How do signatures and approval requirements fo...,"Based on the provided context, signatures and ...",None,The signatures and approval requirements for F...,1,1,0,6.285583,7bd63129-f2e5-490f-9476-6c22bd6c6f06,d448a70c-f110-4feb-abaf-5e1c93e7d651
5,How does the application processing of the FAF...,The application processing of the FAFSA ensure...,None,The application processing of the FAFSA involv...,1,1,0,7.786746,5cd2ade6-2222-4cb6-9e11-bd5a6c6d6a3d,5bc19156-e5f2-4c45-9495-306b8ec4ceab
6,How does the disclosure of Federal Tax Informa...,The disclosure of Federal Tax Information (FTI...,None,The disclosure of Federal Tax Information (FTI...,1,1,0,5.761187,117185f1-f8f3-49e3-be64-448c917edf3e,35d44eea-8c13-4c62-9602-bcd166ad4e90
7,"How does the FAFSA Simplification Act, particu...","The FAFSA Simplification Act, through the impl...",None,"The FAFSA Simplification Act, through the impl...",1,1,0,3.915207,f75ab0a3-8ca0-44e0-bab4-7711e8684ec7,b0967eca-ae90-4543-9387-619fb8d1bb7b
8,How do I correctly fill out the FASFA form and...,I don't know.,None,The context provided does not mention the FAFS...,1,0,0,0.788869,08860e29-20c0-4138-b868-71c81568e525,d37d06f2-4004-4540-8210-e0b2b95ac4ee
9,How do I correctly interpret the ISIR in the F...,"Based on the provided context, to correctly in...",None,The ISIR is an output document produced after ...,1,1,0,7.597327,5e30fef0-f4f8-4161-9667-2121bbb73403,7360618d-99dc-409d-accd-ff0e2cb7d47c


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [31]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [32]:
rag_documents = docs

In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

##### ✅ Answer:

Modifying the chunk size directly impacts how documents are split and retrieved in a RAG (Retrieval-Augmented Generation) pipeline. If chunks are **too small**, they may lack enough context for accurate retrieval, leading to irrelevant or incomplete answers. If chunks are **too large**, they might include unrelated information, increase token usage, or exceed the model’s context window.

Optimizing chunk size helps balance:
- **Contextual relevance** (enough information to answer a query),
- **Retrieval accuracy** (precision in matching queries to documents), and
- **Efficiency** (fitting within token limits and reducing processing time).

Finding the right chunk size improves both the **quality** and **speed** of responses in the QA system.


In [34]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

##### ✅ Answer:

The embedding model is responsible for converting text chunks and user queries into numerical vectors that capture their semantic meaning. Changing the embedding model can significantly affect the **accuracy of document retrieval** in a RAG pipeline.

Different embedding models vary in:
- **Semantic understanding**: Some models better capture meaning, synonyms, and context.
- **Domain specialization**: Certain models are fine-tuned for legal, medical, or technical content.
- **Vector space alignment**: Embeddings must be aligned for queries and documents to match effectively.
- **Performance and latency**: Larger models may be more accurate but slower or more resource-intensive.

Using a more powerful or better-suited embedding model can lead to **more relevant retrieved chunks**, which in turn results in **better LLM responses** and improved overall system performance.


In [35]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [36]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [37]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [38]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question. Based on the information you've shared, it sounds like navigating loan options can feel overwhelming, and I'm here to help clarify things for you.\n\nThe types of loans mentioned in the context include:\n\n1. **Direct Subsidized Loans** – These loans are based on the student's financial need. Interest is paid by the government while the student is in school at least half-time.\n\n2. **Direct Unsubsidized Loans** – These loans are available regardless of financial need, and interest accrues while the student is in school.\n\n3. **Direct PLUS Loans** – These are loans that a parent of a dependent student can take out to help pay for the student's cost of attendance (COA), assuming eligibility. There isn’t a fixed loan limit, but it cannot exceed the student’s COA minus other financial aid.\n\nAdditionally, it is noted that students may qualify for a combination of loan types—for example, both Direct Subsidized and Unsubsidized Loans, particularly if a depend

Finally, we can evaluate the new chain on the same test set!

In [40]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'shiny-knowledge-35' at:
https://smith.langchain.com/o/8a526e26-7800-4aae-98b7-c8c0ecc66e3c/datasets/5c5d3f15-0802-4af3-81c0-92b099c02df1/compare?selectedSessions=cd581e89-f29a-4c3f-8602-d96b780867ff




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,Wht is the Title IV?,Thank you for your question! Based on the cont...,None,The context explains that Title IV involves pr...,0,0,1,2.473798,4d38a461-c9c6-493b-ae66-878cef366f7b,23ba65da-b99c-460b-a632-79690db4f182
1,"Student live with disabled sibling and nephew,...",Thank you for your question — I understand how...,None,"Based on the context, the student and their di...",0,0,1,3.662275,d4a1474b-9775-4f77-a4ce-a726935b8c29,71ab37e3-1049-4429-83be-fe144c8d9db5
2,How does the ISIR relate to the FTI disclosure...,Thank you for your thoughtful question about t...,None,The ISIR (Institutional Student Information Re...,1,1,1,7.671439,1a24a6c5-daac-4315-97bb-aa801aead613,0c24ab33-a5ec-4e7d-85eb-f933f5e2bf67
3,How does the Federal Student Aid Information C...,I understand how confusing and frustrating it ...,None,The Federal Student Aid Information Center (FS...,1,1,1,4.750203,130a05f8-0a6c-48a3-84d5-a6bf297774ad,729e6665-8779-4570-9473-ff40d60f1b16
4,How do signatures and approval requirements fo...,Thank you for your thoughtful question. I unde...,None,The signatures and approval requirements for F...,1,1,1,4.490880,7bd63129-f2e5-490f-9476-6c22bd6c6f06,4de3a88b-ec12-45df-8faa-d59ada3165fb
5,How does the application processing of the FAF...,Thank you for your thoughtful question! It’s c...,None,The application processing of the FAFSA involv...,1,1,1,9.176234,5cd2ade6-2222-4cb6-9e11-bd5a6c6d6a3d,48e31d91-6721-49a9-a84c-5115dee9778f
6,How does the disclosure of Federal Tax Informa...,Thank you for your thoughtful question. Based ...,None,The disclosure of Federal Tax Information (FTI...,1,1,1,5.885231,117185f1-f8f3-49e3-be64-448c917edf3e,adc2d733-874b-400c-8487-e630d7a2b0ae
7,"How does the FAFSA Simplification Act, particu...",Thank you for your thoughtful question. I unde...,None,"The FAFSA Simplification Act, through the impl...",1,1,1,8.620814,f75ab0a3-8ca0-44e0-bab4-7711e8684ec7,d987a646-deb6-44fc-80dd-01c8ee739e8c
8,How do I correctly fill out the FASFA form and...,Thank you for your question—it’s really import...,None,The context provided does not mention the FAFS...,0,0,1,5.061467,08860e29-20c0-4138-b868-71c81568e525,f8267ee0-1baa-49ee-bca9-5cf609daec90
9,How do I correctly interpret the ISIR in the F...,Thank you for your thoughtful question about i...,None,The ISIR is an output document produced after ...,1,1,1,5.749026,5e30fef0-f4f8-4161-9667-2121bbb73403,09c1c0de-e162-4fa2-8f57-d526cf4a3101


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

##### ✅ Answer:
![Comparison Results](./images/loan_synthetic_data_comparison.png)

![Comparison Results](./images/comparison_numbers.png)


### 📊 Result Comparison Summary

| Metric         | `rag_chain` | `empathy_rag_chain` | Change                |
|----------------|-------------|----------------------|------------------------|
| **Correctness**  | 0.9167      | 0.75                 | 🔻 Decrease            |
| **Empathy**      | 0.00        | 1.00                 | 🔺 Dramatic Increase   |
| **Helpfulness**  | 0.8333      | 0.75                 | 🔻 Slight Decrease     |
| **Tokens Used**  | 40,430      | 24,210               | 🔻 Decreased by 40%    |

### 🔍 Analysis of Differences Between `rag_chain` and `empathy_rag_chain`

| Factor               | `rag_chain` Setting                             | `empathy_rag_chain` Setting                      | Potential Impact on Results |
|----------------------|--------------------------------------------------|--------------------------------------------------|------------------------------|
| **Chunk Size**       | `chunk_size = 500`                               | `chunk_size = 1000`                              | Larger chunks reduce the number of chunks, lowering token usage and potentially improving context coherence. However, this can dilute focus and harm **Correctness** and **Helpfulness**. |
| **Prompt Template**  | Neutral factual prompt                           | Empathy-driven prompt                            | Adding empathy may improve emotional tone (**Empathy ↑**) but reduce factual clarity, hurting **Correctness** and **Helpfulness** if answers stray from source. |
| **Embedding Model**  | `text-embedding-3-small`                         | `text-embedding-3-large`                         | The larger model provides richer semantic embeddings, which may enhance empathetic or nuanced retrieval but could pull in broader, less precise content. |

---

 **Conclusion**: The changes made in `empathy_rag_chain` optimized for user connection at the cost of factual precision. Adjustments to chunk size, prompt design, and embedding richness together contributed to the observed metric shifts.
